In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
import pandas as pd
data=pd.read_csv('../data/Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [15]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### ANN Regression Problem statement

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

In [17]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard and EarlyStopping callbacks
log_dir = "../regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 100230.2500 - mae: 100230.2500 - val_loss: 98538.3594 - val_mae: 98538.3594
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 100150.0312 - mae: 100150.0312 - val_loss: 97096.1562 - val_mae: 97096.1562
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 98860.9922 - mae: 98860.9922 - val_loss: 93383.8828 - val_mae: 93383.8828
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 93669.3281 - mae: 93669.3281 - val_loss: 87138.1094 - val_mae: 87138.1094
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 87756.3438 - mae: 87756.3438 - val_loss: 78957.9766 - val_mae: 78957.9766
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 78623.3047 - mae: 78623.3047 - val_loss: 70200.5781 - val_mae: 70200.5781
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 68515.6484 - mae: 68515.6484 - val_loss: 62387.3828 - val_mae: 62387.3828
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/st

In [21]:
# Save the model
model.save('../model/regression_model.h5')

In [22]:
## Evaluate model on the test data
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

# test metrics
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print(f"Test Mean Squared Error: {mse_test:.2f}")
print(f"Test Mean Absolute Error: {mae_test:.2f}")
print(f"Test R² Score: {r2_test:.2f}")

# train metrics
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
print(f"Train Mean Squared Error: {mse_train:.2f}")
print(f"Train Mean Absolute Error: {mae_train:.2f}")
print(f"Train R² Score: {r2_train:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Test Mean Squared Error: 3403880270.33
Test Mean Absolute Error: 50366.05
Test R² Score: -0.03
Train Mean Squared Error: 3316145891.97
Train Mean Absolute Error: 49461.44
Train R² Score: -0.00


### Interestingly, while the ANN regression model shows no signs of overfitting—as indicated by similar training and test MAE—the overall performance remains poor, with a high mae (~50,256) and a negative R² score (-0.03). This suggests that the model is consistently underperforming, not due to model complexity, but likely due to the lack of meaningful correlation between input features and the target variable (EstimatedSalary). The result highlights that even powerful models like ANN cannot compensate for weak or irrelevant features.

In [23]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
%tensorboard --logdir ../regressionlogs/fit/20250708-200727  # use your own log directory here

Reusing TensorBoard on port 6009 (pid 10480), started 0:00:58 ago. (Use '!kill 10480' to kill it.)